<a href="https://colab.research.google.com/github/RodolphoSartori/Macro/blob/main/PIM_PF_se%C3%A7%C3%B5es_e_atividades_industriais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [ ]:
!pip install sidrapy
!pip install pandas
!pip install plotly

import sidrapy as sidra
import pandas as pd
import plotly.express as px

#Coleta

In [ ]:
#Coleta de Dados
dados = pd.read_json(path_or_buf = "https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/11604/p/all/c544/129314,129315,129316/d/v11604%201?formato=json")
dados

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Mês (Código),Mês,Seções e atividades industriais (CNAE 2.0) (Có...,Seções e atividades industriais (CNAE 2.0)
1,1,Brasil,,,...,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,200201,janeiro 2002,129314,1 Indústria geral
2,1,Brasil,,,...,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,200201,janeiro 2002,129315,2 Indústrias extrativas
3,1,Brasil,,,...,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,200201,janeiro 2002,129316,3 Indústrias de transformação
4,1,Brasil,,,...,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,200202,fevereiro 2002,129314,1 Indústria geral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,1,Brasil,2,%,2.1,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,202411,novembro 2024,129315,2 Indústrias extrativas
825,1,Brasil,2,%,3.2,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,202411,novembro 2024,129316,3 Indústrias de transformação
826,1,Brasil,2,%,3.1,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,202412,dezembro 2024,129314,1 Indústria geral
827,1,Brasil,2,%,0.0,1,Brasil,11604,PIMPF - Variação acumulada em 12 meses (em rel...,202412,dezembro 2024,129315,2 Indústrias extrativas


#Tratamento dos Dados

In [ ]:
dados_tratados = dados [["V", "D3C", "D4N"]]
dados_tratados = dados_tratados.drop(index=0)
dados_tratados = dados_tratados.rename(columns={"V": "Valor", "D3C": "Data", "D4N" : "Categorias"})
#dados_tratados.dtypes- usar para ver em qual formato estão meus dados
dados_tratados["Data"] = dados_tratados["Data"].str.split(' ').str[0] #sempre separar os números na coluna Data
dados_tratados["Data"] = pd.to_datetime(dados_tratados["Data"], format="%Y%m") #informar que a coluna data está em Ym para conseguir usar datetime
dados_tratados['Valor'] = dados_tratados['Valor'].replace('...', pd.NA) #não posso deixar reticências na coluna valor
dados_tratados["Valor"] = pd.to_numeric(dados_tratados["Valor"])
dados_tratados["Categorias"] = dados_tratados["Categorias"].str.replace(r'^\d+\s+', '', regex=True) #\d ESPAÇO remove número
dados_tratados = dados_tratados.set_index("Data")
dados_tratados

,Valor,Categorias
Data,,
2002-01-01,NaN,Indústria geral
2002-01-01,NaN,Indústrias extrativas
2002-01-01,NaN,Indústrias de transformação
2002-02-01,NaN,Indústria geral
2002-02-01,NaN,Indústrias extrativas
...,...,...
2024-11-01,2.1,Indústrias extrativas
2024-11-01,3.2,Indústrias de transformação
2024-12-01,3.1,Indústria geral


#Visualização

In [ ]:
fig = px.line(dados_tratados,
              x=dados_tratados.index,
              y="Valor",
              color="Categorias",
              title="PIMPF - Variação acumulada em 12 meses")


fig.update_layout(
    title={
        'text': "PIMPF - Variação acumulada em 12 meses",
        'y':0.97,                  # Posição vertical do título (0 = base, 1 = topo)
        'x':0.4,                  # Posição horizontal do título (0 = esquerda, 1 = direita)
        'xanchor': 'center',      # Ancoragem horizontal ('auto', 'center', 'left', 'right')
        'yanchor': 'top',        # Ancoragem vertical ('auto', 'top', 'middle', 'bottom')
        'font': dict(size=20)    # Tamanho da fonte do título
    }
)

In [ ]:
categorias_unicas = dados_tratados['Categorias'].unique()

for categoria in categorias_unicas:
    dados_categoria = dados_tratados[dados_tratados['Categorias'] == categoria]
    fig = px.line(dados_categoria,
                  x=dados_categoria.index,
                  y="Valor",
                  title=f"PIMPF - Variação acumulada em 12 meses - {categoria}")  # Título com a categoria

    fig.update_layout(
        title={
            'y':0.97,
            'x':0.4,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20)
        },
        xaxis_title="Data",
        yaxis_title="Valor"
    )

    fig.show()  # Movido para dentro do loop